

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# Clean install in Colab
%pip uninstall -y unsloth unsloth_zoo trl
%pip install --upgrade --force-reinstall --no-cache-dir "trl>=0.14,<0.17" unsloth unsloth_zoo

# IMPORTANT: After installs, Colab needs a restart for imports to use the new wheels
import IPython, sys
print("Installed. Python:", sys.version)
print("Please now go to: Runtime -> Restart runtime, then re-run the next cell.")


/home/rgmatr1x/hetal/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
/home/rgmatr1x/hetal/.venv/bin/python: No module named pip
Note: you may need to restart the kernel to use updated packages.
Installed. Python: 3.12.11 (main, Aug  8 2025, 17:06:48) [Clang 20.1.4 ]
Please now go to: Runtime -> Restart runtime, then re-run the next cell.


In [ ]:
import sys
print("Python:", sys.version)

import unsloth, transformers, trl
print("unsloth =", getattr(unsloth, "__version__", "?"))
print("transformers =", transformers.__version__)
print("trl =", trl.__version__)

# Optional: confirm the RL replacement map includes the needed key
import importlib
rl = importlib.import_module("unsloth.models.rl")
print("has align_logprobs_with_mask:", "align_logprobs_with_mask" in rl.RL_REPLACEMENTS)

Python: 3.12.11 (main, Aug  8 2025, 17:06:48) [Clang 20.1.4 ]
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/rgmatr1x/hetal/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[torchao|WARNING]Skipping import of cpp extensions due to incompatible torch version 2.8.0+cu128 for torchao version 0.14.1             Please see https://github.com/pytorch/ao/issues/2919 for more info


🦥 Unsloth Zoo will now patch everything to make training faster!
unsloth = 2025.10.12
transformers = 4.57.1
trl = 0.23.0
has align_logprobs_with_mask: True


## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    NVIDIA GeForce RTX 4080 SUPER. Num GPUs = 1. Max memory: 15.548 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu128. CUDA: 8.9. CUDA Toolkit: 12.8. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(50000))      # Use the first 5,000 for training
validation_dataset = shuffled_dataset.select(range(50000, 55000)) # Use the next 500 for validation

In [ ]:
# The instructional prompt template for training
training_prompt = """You are an expert mathematician. Verify if this solution correctly answers the question.

Question:
{question}

Solution:
{solution}

Is the solution correct? Check the mathematical reasoning, calculations, and final answer.
Respond with only 'True' or 'False'.

Answer: {output}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(
            question=question,
            solution=str(solution),
            output=str(output)
        ) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(
    formatting_prompts_func,
    batched=True
)

Map: 100%|██████████| 50000/50000 [00:01<00:00, 34499.61 examples/s]


## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 128, # A common practice is to set alpha = 2 * r
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth 2025.10.12 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


# Implementing Learning Rate Finder to Determine Optimal Learning Rate for Training

In [ ]:
def lr_finder_forward_only(
    model,
    train_dataset,
    tokenizer,
    dataset_text_field="text",
    batch_size=1,
    min_lr=1e-7,
    max_lr=1e-1,
    num_iter=50,
    max_seq_length=512,
    device=None,
):
    """
    Lightweight Learning Rate Finder for language model fine-tuning.
    Runs forward-only passes (no backprop) to efficiently estimate an ideal LR range.
    """

    import torch, math
    import matplotlib.pyplot as plt
    from torch.utils.data import DataLoader
    from tqdm.auto import tqdm
    import numpy as np

    # --- DEVICE SETUP ---
    # Use GPU if available, else fall back to CPU.
    if device is None:
        device = "cuda" if torch.cuda.is_available() else "cpu"

    # Set model to eval mode and clear any leftover GPU cache.
    model.eval()
    torch.cuda.empty_cache()

    # DATA SUBSAMPLING
    # Use a smaller subset of the dataset for speed.
    # The subset size ensures we have enough batches for the given num_iter.
    subset = train_dataset.select(range(min(len(train_dataset), num_iter * batch_size)))

    # TOKENIZATION FUNCTION
    # Custom collate_fn used by DataLoader to tokenize on-the-fly.
    def collate_fn(batch):
        # Extract text field from dataset entries
        texts = [ex[dataset_text_field] for ex in batch]

        # Tokenize with truncation + padding to uniform length
        toks = tokenizer(
            texts,
            truncation=True,
            padding=True,
            max_length=max_seq_length,
            return_tensors="pt",
        )

        # For language modeling: labels are the same as input_ids (next-token prediction)
        toks["labels"] = toks["input_ids"].clone()
        return toks

    # DATALOADER
    # Shuffles the subset and batches it for the LR sweep
    dataloader = DataLoader(subset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    # LR RANGE SETUP
    # Compute multiplicative LR step factor for exponential growth between min_lr and max_lr
    lr_mult = (max_lr / min_lr) ** (1 / num_iter)
    # Precompute the LR schedule (logarithmically spaced)
    lrs = [min_lr * (lr_mult ** i) for i in range(num_iter)]

    # TRACKING VARIABLES
    losses, avg_loss, best_loss = [], 0.0, float("inf")

    # MAIN LR SWEEP LOOP
    for i, batch in enumerate(tqdm(dataloader, total=num_iter)):
        if i >= num_iter:
            break  # stop after reaching desired iterations

        # Move batch tensors to GPU/CPU
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass only — no optimizer step or gradient computation
        with torch.inference_mode():
            outputs = model(**batch)
            loss = outputs.loss.item()

        # Smooth the loss to make the LR curve cleaner
        avg_loss = 0.05 * loss + 0.95 * avg_loss if i > 0 else loss
        losses.append(avg_loss)

        # Stop early if the loss diverges (indicates LR is too high)
        if avg_loss > 10 * best_loss:
            print(f"Stopping early (loss exploded at step {i})")
            break

        # Track best (minimum) loss so far
        best_loss = min(best_loss, avg_loss)

        # Occasionally clear GPU cache to prevent memory buildup
        if i % 20 == 0:
            torch.cuda.empty_cache()

    # BEST LR SUGGESTION
    # Heuristic: find LR at steepest downward slope (min gradient of log(loss))
    def suggest_lr(lrs, losses):
        losses = np.array(losses)
        grads = np.gradient(np.log(losses))
        min_grad_idx = np.argmin(grads)
        return lrs[min_grad_idx]

    best_lr = suggest_lr(lrs[:len(losses)], losses)

    # VISUALIZATION
    plt.figure(figsize=(7,5))
    plt.plot(lrs[:len(losses)], losses)
    plt.xscale("log")
    plt.xlabel("Learning rate (log scale)")
    plt.ylabel("Smoothed loss")
    plt.title(f"LR Finder (suggested LR ≈ {best_lr:.2e})")
    plt.show()

    # Print and return results
    print(f"Suggested LR ≈ {best_lr:.2e}")
    return lrs[:len(losses)], losses, best_lr


In [ ]:
# RUN LR FINDER
gc.collect(); torch.cuda.empty_cache()

lrs, losses, best_lr = lr_finder_forward_only(
    model=model,
    train_dataset=formatted_train_dataset_small,
    tokenizer=tokenizer,
    dataset_text_field="text",
    batch_size=2,
    num_iter=2500,
    max_seq_length=256,



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

formatted_val_dataset = validation_dataset.map(
    formatting_prompts_func,
    batched=True
)

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    eval_dataset = formatted_val_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 8,
        warmup_steps = 50,
        max_steps = 500,
        #num_train_epochs = 3,
        learning_rate = 3e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 50,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "reduce_lr_on_plateau",
        metric_for_best_model = "eval_loss",
        seed = 42,
        output_dir = "outputs",
        report_to = "none",
        eval_strategy = "steps",  # Enable evaluation
        eval_steps = 50,
        lr_scheduler_kwargs = {"factor":0.5, "patience":2},
    ),
)

Unsloth: Tokenizing ["text"] (num_proc=20): 100%|██████████| 5000/5000 [00:02<00:00, 2170.32 examples/s]


## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕


In [ ]:
trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 50,000 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 8 x 1) = 32
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,Validation Loss
50,0.849200,0.723802
100,0.724700,0.708016
150,0.712700,0.697831
200,0.700800,0.689693
250,0.677300,0.682319
300,0.675600,0.677954
350,0.678400,0.671034
400,0.670100,0.665497
450,0.666300,0.659942
500,0.649900,0.654381


Unsloth: Not an error, but LlamaForCausalLM does not accept `num_items_in_batch`.
Using gradient accumulation will be very slightly less accurate.
Read more on gradient accumulation issues here: https://unsloth.ai/blog/gradient


TrainOutput(global_step=500, training_loss=0.700503059387207, metrics={'train_runtime': 10043.0912, 'train_samples_per_second': 1.593, 'train_steps_per_second': 0.05, 'total_flos': 3.104460913878958e+17, 'train_loss': 0.700503059387207, 'epoch': 0.32})


## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
from tqdm import tqdm

# Prepare the model for inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are an expert mathematician. Verify if this solution correctly answers the question.

Question:
{question}

Solution:
{solution}

Is the solution correct? Check the mathematical reasoning, calculations, and final answer.
Respond with only 'True' or 'False'.

Answer:"""

correct = 0
total = 0
train_size = len(train_dataset)

# Select a sample from the validation set
validation_dataset = shuffled_dataset.select(range(train_size+1000, train_size+2000))

# Loop through the entire validation dataset
for example in tqdm(validation_dataset):
    question = example["question"]
    solution = example["solution"]
    true_label = str(example["is_correct"]).strip().lower()  # "true" or "false"

    # Format the INFERENCE prompt (without answer)
    prompt = inference_prompt.format(
        question=question,
        solution=str(solution)
    )

    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate model prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response = tokenizer.batch_decode(outputs)[0]

    # Extract the model's answer after "Answer:"
    pred_text = response.split("Answer:")[-1].strip()

    # Remove the end-of-text token
    pred_text = pred_text.replace('<|end_of_text|>', '').strip().lower()

    # Match against true label
    if pred_text.startswith(true_label):
        correct += 1

    total += 1

# Compute accuracy
accuracy = correct / total * 100
print(f"\n✓ Model accuracy on {total} samples: {accuracy:.2f}%")
print(f"Correct predictions: {correct}/{total}")

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are an expert mathematician. Verify if this solution correctly answers the question.

Question:
{question}

Solution:
{solution}

Is the solution correct? Check the mathematical reasoning, calculations, and final answer.
Respond with only 'True' or 'False'.

Answer:"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question=question, solution=str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
A suitcase lock has 3 dials with the digits $0, 1, 2,..., 9$ on each. How many different settings are possible if all three digits have to be different?

#### SOLUTION ####

This is easy to calculate in python. Let's generate all possible combinations and filter them by checking if there are repeating digits.
<llm-code>
from itertools import permutations

# calculate all permutations of 0, 1, 2, 3, 4, 5, 6, 7, 8, 9
permuted_digits = permutations([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], 3)

# Create an empty array to store the valid permutations
valid_permutations = []

# Loop over the permuted digits and check if they have repeating digits
for i in permuted_digits:
    if i[0] != i[1] and i[0] != i[2] and i[1] != i[2]:
        valid_permutations.append(i)

# Count the number of valid permutations
number_of_valid_permutations = len(valid_permutations)
number_of_valid_permutations
</llm-code>
<llm-code-output>
720
</llm-code-output>
So the answer is $\boxed{720}$.

#### MODEL'S

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import pandas as pd
from tqdm import tqdm
'''
Testing and submission generation
'''
inference_prompt = """You are an expert mathematician. Verify if this solution correctly answers the question.

Question:
{question}

Solution:
{solution}

Is the solution correct? Check the mathematical reasoning, calculations, and final answer.
Respond with only 'True' or 'False'.

Answer:"""

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []


# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the INFERENCE prompt (without answer)
    prompt = inference_prompt.format(
        question=question,
        solution=str(solution)
    )

    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Extract the model's answer after "Answer:"
    pred_text = response_text.split("Answer:")[1]

    # Remove the end-of-text token
    pred_text = pred_text.replace('<|end_of_text|>', '').strip().lower()

    if pred_text == "true":
        prediction = True
    else:
        prediction = False

    # Parse the prediction and add it to our list
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

100%|██████████| 10000/10000 [20:44<00:00,  8.03it/s]


Submission file 'submission.csv' created successfully!
You can now download this file and submit it to the Kaggle competition.


# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Save to local directory instead of Drive
save_path = "./llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

Model checkpoint and tokenizer saved to: ./llama3_8b_math_verifier_checkpoint


In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
'''
Testing and submission generation
'''

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

train_size=len(train_dataset)

# Select a sample from the validation set
validation_dataset = shuffled_dataset.select(range(train_size+1000, train_size+1010))

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the INFERENCE prompt (without answer)
    prompt = inference_prompt.format(
        question=question,
        solution=str(solution)
    )

    inputs = tokenizer(
        [prompt],
        return_tensors="pt"
    ).to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Extract the model's answer after "Answer:"
    pred_text = response_text.split("Answer:")[1]

    # Remove the end-of-text token
    pred_text = pred_text.replace('<|end_of_text|>', '').strip().lower()

    if pred_text == "true":
        prediction = True
    else:
        prediction = False

    # Parse the prediction and add it to our list
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

 16%|█▌        | 1562/10000 [15:08<1:21:47,  1.72it/s]


KeyboardInterrupt: 